In [1]:
from collections import defaultdict
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import statsmodels.formula.api as smf

from data import load_pcd_df
from plotting import save_plot

pio.templates.default = "plotly_white"

In [2]:
results_dir = 'results/2025-02-21_fresh_notebook/'
os.makedirs(results_dir, exist_ok=True)

In [3]:
save = True
benchmarks = ['MMLU', 'GPQA Diamond', 'MATH-500', 'MATH 5', 'HumanEval', 'LMSys Chatbot Arena ELO']
benchmark_is_mqa = {'MMLU': True, 'GPQA Diamond': True, 'MATH 5': False, 'MATH-500': False, 'HumanEval': False, 'LMSys Chatbot Arena ELO': False}
min_num_data_points_for_regression = 4

# Using Artificial Analysis data

## Load Artificial Analysis dataset

In [4]:
aa_df = pd.read_csv('data/aa_data_with_math5.csv')
aa_df

,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH-500,HumanEval,LMSys Chatbot Arena ELO,MATH 5
0,Claude-3-Haiku,2024-03,0.5000,122.7,0.467,1000.0,71,33,39.0,77.0,1179.0,13.0
1,Claude-3-Opus,2024-03,30.0000,26.5,1.984,1000.0,84,50,64.0,83.0,1248.0,34.0
2,Claude-3-Sonnet,2024-03,6.0000,61.8,0.789,1000.0,77,37,41.0,71.0,1201.0,16.0
3,Claude-3.5-Haiku,2024-06,1.6000,64.2,0.768,1000.0,81,37,67.0,87.0,1236.0,NaN
4,Claude-3.5-Sonnet-2024-06,2024-06,6.0000,55.9,0.906,1000.0,88,56,71.0,90.0,1268.0,46.0
5,Claude-3.5-Sonnet-2024-10,2024-10,6.0000,55.2,0.907,1000.0,89,58,76.0,96.0,1282.0,53.0
6,Gemini 2.0 Flash,2025-02,0.1750,NaN,NaN,NaN,88,62,93.0,90.0,1358.0,82.0
7,Gemini-1.5-Flash-2024-05,2024-05,0.1300,298.4,0.307,1000.0,79,39,55.0,NaN,1227.0,23.0
8,Gemini-1.5-Flash-2024-09,2024-09,0.1300,190.5,0.348,1000.0,75,45,83.0,83.0,1271.0,58.0
9,Gemini-1.5-Flash-8B,2024-10,0.0700,285.2,0.335,1000.0,75,30,70.0,12.0,1211.0,NaN


In [5]:
# 'Release Date' is a string with the format 'YYYY-MM'
aa_df['Release Date'] = pd.to_datetime(aa_df['Release Date'].str.strip(), format='%Y-%m')

## Load Epoch AI price dataset

In [6]:
api_price_df = pd.read_csv('data/epoch_ai_price_data_not_in_aa_with_benchmarks.csv')
api_price_df['Release Date'] = pd.to_datetime(api_price_df['Release Date'].str.strip(), format='%Y-%m-%d')
api_price_df

,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH 5,MATH-500,HumanEval,LMSys Chatbot Arena ELO
0,Claude 2,2024-08-12,12.0000,NaN,NaN,NaN,78.5,35.0,10.0,NaN,NaN,NaN
1,Claude 2.1,2024-08-12,12.0000,NaN,NaN,NaN,NaN,36.0,11.0,NaN,NaN,NaN
2,Claude Instant,2024-08-12,1.2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cohere Command,2024-08-13,1.6250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cohere Command Light,2024-08-13,0.3750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Command R,2024-08-13,0.7500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1180.0
6,Command R+,2024-08-13,6.0000,NaN,NaN,NaN,75.7,NaN,NaN,NaN,NaN,1215.0
7,Command R+,2024-09-13,4.3750,NaN,NaN,NaN,75.7,NaN,NaN,NaN,NaN,1215.0
8,DeepSeek-Coder-V2 236B,2024-09-11,0.1750,NaN,NaN,NaN,79.2,NaN,NaN,NaN,NaN,1178.0
9,DeepSeek-R1,2025-01-20,0.9600,NaN,NaN,NaN,NaN,71.7,93.1,NaN,NaN,1362.0


In [7]:
# Merge the two datasets
aa_df = pd.concat([aa_df, api_price_df])
aa_df.sort_values(by='Release Date', inplace=True)
# Reset the index
aa_df.reset_index(drop=True, inplace=True)
aa_df

,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH-500,HumanEval,LMSys Chatbot Arena ELO,MATH 5
0,GPT-3 175B (davinci),2021-11-20,60.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
1,GPT-3 175B (davinci),2022-08-31,60.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
2,GPT-3 175B (davinci),2022-09-01,20.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
3,GPT-3.5,2022-11-30,20.000,NaN,NaN,NaN,64.8,NaN,NaN,NaN,NaN,NaN
4,GPT-4,2023-03-01,37.500,23.6,0.724,1000.0,86.0,33.0,21.0,67.0,1186.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...
66,Mistral Small 3,2025-01-01,0.475,NaN,NaN,NaN,82.0,46.0,74.0,85.0,1210.0,45.0
67,DeepSeek-R1,2025-01-20,0.960,NaN,NaN,NaN,NaN,71.7,NaN,NaN,1362.0,93.1
68,o3-mini,2025-01-31,1.925,NaN,NaN,NaN,NaN,74.3,NaN,NaN,1306.0,95.2
69,o1-mini,2025-01-31,1.925,NaN,NaN,NaN,NaN,59.5,NaN,NaN,1304.0,84.3


In [8]:
# Rename all instances of 'GPT-3 175B (davinci)' to 'GPT-3'
aa_df.loc[aa_df['Model Name'] == 'GPT-3 175B (davinci)', 'Model Name'] = 'GPT-3'
aa_df


,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH-500,HumanEval,LMSys Chatbot Arena ELO,MATH 5
0,GPT-3,2021-11-20,60.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
1,GPT-3,2022-08-31,60.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
2,GPT-3,2022-09-01,20.000,NaN,NaN,NaN,43.9,NaN,NaN,NaN,NaN,NaN
3,GPT-3.5,2022-11-30,20.000,NaN,NaN,NaN,64.8,NaN,NaN,NaN,NaN,NaN
4,GPT-4,2023-03-01,37.500,23.6,0.724,1000.0,86.0,33.0,21.0,67.0,1186.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...
66,Mistral Small 3,2025-01-01,0.475,NaN,NaN,NaN,82.0,46.0,74.0,85.0,1210.0,45.0
67,DeepSeek-R1,2025-01-20,0.960,NaN,NaN,NaN,NaN,71.7,NaN,NaN,1362.0,93.1
68,o3-mini,2025-01-31,1.925,NaN,NaN,NaN,NaN,74.3,NaN,NaN,1306.0,95.2
69,o1-mini,2025-01-31,1.925,NaN,NaN,NaN,NaN,59.5,NaN,NaN,1304.0,84.3


In [9]:
aa_df['Model Name and Date'] = aa_df['Model Name'] + ' (' + aa_df['Release Date'].dt.strftime('%Y-%m') + ')'
aa_df['Model Name and Date']

0                GPT-3 (2021-11)
1                GPT-3 (2022-08)
2                GPT-3 (2022-09)
3              GPT-3.5 (2022-11)
4                GPT-4 (2023-03)
                 ...            
66     Mistral Small 3 (2025-01)
67         DeepSeek-R1 (2025-01)
68             o3-mini (2025-01)
69             o1-mini (2025-01)
70    Gemini 2.0 Flash (2025-02)
Name: Model Name and Date, Length: 71, dtype: object

## Explore the data

### Performance

In [10]:
top_n = 1
aa_df = aa_df.sort_values(by='Release Date')

top_models_df_lookup = {}
for benchmark in benchmarks:
    # Get the top-1 model by benchmark score at each point in time
    ever_top_n_models = set()
    unique_dates = aa_df['Release Date'].sort_values().unique()
    for date in unique_dates:
        df_up_to_date = aa_df[aa_df['Release Date'] <= date]
        top_n_models = df_up_to_date.nlargest(top_n, benchmark)
        top_n_model_names = top_n_models['Model Name and Date'].tolist()
        ever_top_n_models.update(top_n_model_names)
    ever_top_n_list = list(ever_top_n_models)
    print(ever_top_n_list)
    top_models_df = aa_df[aa_df['Model Name and Date'].isin(ever_top_n_list)]
    top_models_df_lookup[benchmark] = top_models_df

    # Create base scatter plot with all data
    fig = px.scatter(aa_df, x='Release Date', y=benchmark, 
                    hover_data=['Model Name'], 
                    title=f'{benchmark} Performance over time',
                    color_discrete_sequence=['lightgray'])

    # Add top-1 models in a different color
    fig.add_scatter(x=top_models_df['Release Date'], 
                    y=top_models_df[benchmark],
                    mode='lines+markers',
                    name='Top Model',
                    hovertemplate='%{text}<extra></extra>',
                    text=top_models_df['Model Name'],
                    line=dict(color='red'))
    fig.update_layout(
        width=800,
        height=400,
    )
    fig.show()

['Claude-3.5-Sonnet-2024-06 (2024-06)', 'GPT-4o-2024-08 (2024-08)', 'GPT-4 Turbo (2023-11)', 'GPT-4 (2023-03)', 'GPT-3.5 (2022-11)', 'GPT-3 (2021-11)']


['GPT-4o-2024-05 (2024-05)', 'Claude-3.5-Sonnet-2024-06 (2024-06)', 'o1 (2024-12)', 'o1-preview (2024-09)', 'GPT-4 Turbo (2023-11)', 'GPT-4 (2023-03)', 'GPT-3 (2021-11)', 'Gemini-1.5-Pro-2024-09 (2024-09)']


['GPT-4o-2024-05 (2024-05)', 'GPT-4o-2024-08 (2024-08)', 'Gemini 2.0 Flash (2025-02)', 'GPT-4 Turbo (2023-11)', 'GPT-4 (2023-03)', 'GPT-3 (2021-11)', 'Gemini-1.5-Pro-2024-09 (2024-09)', 'GPT-3.5 Turbo (2023-06)']


['GPT-4o-2024-05 (2024-05)', 'o1-mini (2024-09)', 'o1 (2024-12)', 'GPT-4 Turbo (2023-11)', 'o3-mini (2025-01)', 'GPT-4 (2023-03)', 'GPT-3 (2021-11)', 'Gemini-1.5-Pro-2024-09 (2024-09)']


['GPT-4o-2024-05 (2024-05)', 'GPT-4 Turbo (2023-11)', 'GPT-4 (2023-03)', 'GPT-3 (2021-11)', 'Claude-3.5-Sonnet-2024-10 (2024-10)', 'GPT-3.5 Turbo (2023-06)']


['GPT-4o-2024-05 (2024-05)', 'GPT-4o-2024-11 (2024-11)', 'GPT-4o-2024-08 (2024-08)', 'GPT-4 Turbo (2023-11)', 'GPT-4 (2023-03)', 'DeepSeek-R1 (2025-01)', 'GPT-3 (2021-11)']


In [11]:
top_models_df_lookup

{'MMLU':                    Model Name Release Date  USD per 1M Tokens  \
 0                       GPT-3   2021-11-20              60.00   
 3                     GPT-3.5   2022-11-30              20.00   
 4                       GPT-4   2023-03-01              37.50   
 10                GPT-4 Turbo   2023-11-01              15.00   
 32  Claude-3.5-Sonnet-2024-06   2024-06-01               6.00   
 39             GPT-4o-2024-08   2024-08-01               4.38   
 
     Tokens per Second  Prefill Latency (s)  Prompt Length (tk)  MMLU  \
 0                 NaN                  NaN                 NaN  43.9   
 3                 NaN                  NaN                 NaN  64.8   
 4                23.6                0.724              1000.0  86.0   
 10               39.2                1.246              1000.0  87.0   
 32               55.9                0.906              1000.0  88.0   
 39               81.0                0.673              1000.0  89.0   
 
     GPQA Diamo

### Price vs. performance

In [12]:
# Plot 'MMLU' vs. 'USD per 1M Tokens'
fig = px.scatter(aa_df, x='MMLU', y='USD per 1M Tokens', hover_data=['Model Name'], title='MMLU vs. USD per 1M Tokens')
fig.update_layout(yaxis_type='log')
fig.show()


In [13]:
aa_df['MMLU price-performance'] = aa_df['MMLU'] / aa_df['USD per 1M Tokens']
# Plot 'MMLU price-performance' vs. 'Date'
fig = px.scatter(aa_df, x='Release Date', y='MMLU price-performance', title='MMLU price-performance over time', hover_data=['Model Name'])
fig.update_layout(yaxis_type='log')
fig.show()


In [14]:
# Construct a list of 'Model Name' values that were ever in the top-n based on 'MMLU price-performance'
top_n = 1
aa_df = aa_df.sort_values(by='Release Date')
ever_top_n_models = set()

unique_dates = aa_df['Release Date'].sort_values().unique()

for date in unique_dates:
    df_up_to_date = aa_df[aa_df['Release Date'] <= date]
    top_n_models = df_up_to_date.nlargest(top_n, 'MMLU price-performance')
    top_n_model_names = top_n_models['Model Name and Date'].tolist()
    ever_top_n_models.update(top_n_model_names)

ever_top_n_list = sorted(ever_top_n_models)
print(ever_top_n_list)

# Plot top-n models over time
fig = px.scatter(aa_df[aa_df['Model Name and Date'].isin(ever_top_n_list)], x='Release Date', y='MMLU price-performance', title='MMLU price-performance over time', hover_data=['Model Name and Date'])
fig.update_layout(yaxis_type='log')
fig.show()

['GPT-3 (2021-11)', 'GPT-3 (2022-09)', 'GPT-3.5 (2022-11)', 'GPT-3.5 Turbo (2023-03)', 'GPT-3.5 Turbo (2023-11)', 'Gemini-1.5-Flash-2024-05 (2024-05)', 'Gemini-1.5-Flash-8B (2024-10)', 'Llama 2-7B (2023-12)', 'Llama-2-Chat-13B (2023-07)', 'Llama-3-Instruct-8B (2024-04)', 'Llama-3.1-Instruct-8B (2024-07)', 'Llama-3.2-Instruct-3B (2024-09)']


## Regression to lowest-priced models above a performance lower bound

### Examples

In [15]:
results_subdir = results_dir + 'aa_cheapest_models_examples/'
os.makedirs(results_subdir, exist_ok=True)

In [16]:
"""
  - Set a performance lower bound
  - Track the running best (top) model
  - At each point in time (at some resolution)
    - Filter to new models published in this time window
    - Filter to models with performance above the lower bound
    - Check if any new model is cheaper than current best
    - If so, update the current best
    - Record the current best model at this time point
"""
bench = 'MMLU'
performance_lower_bound = 80
performance_upper_bound = 90
ts = pd.date_range(start='2020-01-01', end=pd.Timestamp.today(), freq='MS')
cheapest_models = []
current_best = None

for i, t in enumerate(ts):
    # Get models published in this time window
    benchmark_df = aa_df
    if i > 0:
        prev_t = ts[i-1]
        benchmark_df = benchmark_df[(benchmark_df['Release Date'] >= prev_t) & (benchmark_df['Release Date'] < t)]
    else:
        benchmark_df = benchmark_df[benchmark_df['Release Date'] < t]
        
    # Filter for performance
    benchmark_df = benchmark_df[
        benchmark_df[bench].notna() &
        (benchmark_df[bench] >= performance_lower_bound) &
        (benchmark_df[bench] < performance_upper_bound)
    ]
    
    if not benchmark_df.empty:
        # Find cheapest new model
        new_best = benchmark_df.loc[benchmark_df['USD per 1M Tokens'].idxmin()]
        
        # Update current best if new model is cheaper (or if no current best)
        if current_best is None or new_best['USD per 1M Tokens'] < current_best['USD per 1M Tokens']:
            current_best = new_best
            cheapest_models.append(current_best)
            print(t, current_best['Model Name'], current_best[bench], f"${current_best['USD per 1M Tokens']:.2f}")
        # elif new_best['USD per 1M Tokens'] == current_best['USD per 1M Tokens']:
        #     # Choose the model with the highest performance
        #     if new_best[bench] > current_best[bench]:
        #         current_best = new_best
        #         cheapest_models.append(current_best)
        #         print(t, current_best['Model Name'], current_best[bench], f"${current_best['USD per 1M Tokens']:.2f}")


2023-04-01 00:00:00 GPT-4 86.0 $37.50
2023-12-01 00:00:00 GPT-4 Turbo 87.0 $15.00
2024-06-01 00:00:00 Gemini-1.5-Pro-2024-05 86.0 $2.19
2024-07-01 00:00:00 Claude-3.5-Haiku 81.0 $1.60
2024-08-01 00:00:00 GPT-4o-mini 82.0 $0.26
2025-02-01 00:00:00 Phi 4 85.0 $0.12


In [17]:
cheapest_models_df = pd.DataFrame(cheapest_models)
cheapest_models_df.head()

,Model Name,Release Date,USD per 1M Tokens,Tokens per Second,Prefill Latency (s),Prompt Length (tk),MMLU,GPQA Diamond,MATH-500,HumanEval,LMSys Chatbot Arena ELO,MATH 5,Model Name and Date,MMLU price-performance
4,GPT-4,2023-03-01,37.50,23.6,0.724,1000.0,86.0,33.0,21.0,67.0,1186.0,23.0,GPT-4 (2023-03),2.293333
10,GPT-4 Turbo,2023-11-01,15.00,39.2,1.246,1000.0,87.0,50.0,74.0,92.0,1256.0,36.0,GPT-4 Turbo (2023-11),5.800000
26,Gemini-1.5-Pro-2024-05,2024-05-01,2.19,64.8,0.738,1000.0,86.0,46.0,66.0,83.0,1260.0,41.0,Gemini-1.5-Pro-2024-05 (2024-05),39.269406
31,Claude-3.5-Haiku,2024-06-01,1.60,64.2,0.768,1000.0,81.0,37.0,67.0,87.0,1236.0,NaN,Claude-3.5-Haiku (2024-06),50.625000
38,GPT-4o-mini,2024-07-01,0.26,112.2,0.626,1000.0,82.0,43.0,79.0,88.0,1273.0,48.0,GPT-4o-mini (2024-07),315.384615


In [18]:
# Fit a line to the data
cheapest_models_df['price'] = cheapest_models_df['USD per 1M Tokens']
cheapest_models_df['log_price'] = np.log10(cheapest_models_df['USD per 1M Tokens'])
cheapest_models_df['date'] = cheapest_models_df['Release Date'].map(lambda x: pd.Timestamp(x).toordinal())
exponential_model = smf.ols('log_price ~ date', data=cheapest_models_df).fit()
print(exponential_model.summary())

linear_model = smf.ols('price ~ date', data=cheapest_models_df).fit()
print(linear_model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.871
Method:                 Least Squares   F-statistic:                     34.90
Date:                Fri, 21 Feb 2025   Prob (F-statistic):            0.00411
Time:                        14:53:32   Log-Likelihood:               -0.92738
No. Observations:                   6   AIC:                             5.855
Df Residuals:                       4   BIC:                             5.438
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2917.8175    493.886      5.908      0.0

/Users/bencottier/miniconda3/envs/llm-efficiency/lib/python3.12/site-packages/statsmodels/stats/stattools.py:74: ValueWarning:

omni_normtest is not valid with less than 8 observations; 6 samples were given.

/Users/bencottier/miniconda3/envs/llm-efficiency/lib/python3.12/site-packages/statsmodels/stats/stattools.py:74: ValueWarning:

omni_normtest is not valid with less than 8 observations; 6 samples were given.



In [19]:
# Calculate annual rate of decrease
annual_slope = exponential_model.params['date'] * 365  # Convert daily to annual
annual_factor = int(round(10**(-annual_slope)))  # Convert log slope to factor

# Plot the exponential trendline with the data
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=cheapest_models_df['Release Date'],
    y=10**exponential_model.predict(cheapest_models_df['date']),
    mode='lines',
    name=f'Trendline: {annual_factor}x decrease per year',
    line=dict(color='lightgrey', dash='dash')
))
fig.add_trace(go.Scatter(
    x=cheapest_models_df['Release Date'],
    y=cheapest_models_df['USD per 1M Tokens'],
    mode='markers+text',
    name=f'Cheapest models with {performance_lower_bound}-{performance_upper_bound}% on {bench}',
    text=cheapest_models_df['Model Name'],
    textposition='bottom left',
    marker=dict(color='blue')
    # line=dict(shape='hv'),
))
other_models_df = aa_df[(aa_df[bench].notna()) & (aa_df[bench] >= performance_lower_bound) & ~(aa_df['Model Name'].isin(cheapest_models_df['Model Name']))]
fig.add_trace(go.Scatter(
    x=other_models_df['Release Date'],
    y=other_models_df['USD per 1M Tokens'],
    mode='markers+text',
    name=f'Other models with {performance_lower_bound}-{performance_upper_bound}% on {bench}',
    text=other_models_df['Model Name'],
    marker=dict(color='lightblue')
))
fig.update_layout(
    title=f'Price of the cheapest model with {performance_lower_bound}-{performance_upper_bound}% on {bench}'
)
fig.update_traces(textposition='bottom left')
fig.update_layout(yaxis_type='log')
fig.update_layout(xaxis_title='Month')
fig.update_layout(yaxis_title='Price in USD per million tokens')
# Lower the lower x limit
min_date = min(cheapest_models_df['Release Date'].min(), other_models_df['Release Date'].min())
max_date = max(cheapest_models_df['Release Date'].max(), other_models_df['Release Date'].max())
fig.update_layout(xaxis_range=[min_date - pd.Timedelta(days=90), max_date + pd.Timedelta(days=30)])
fig.update_layout(
    width=800,
    height=600,
    font=dict(size=10),
    legend=dict(
        yanchor="top",
        y=0.25,
        xanchor="right",
        x=0.4,
        bordercolor="lightgrey",
        borderwidth=1
    )
)
if save:
    save_plot(fig, results_dir + 'aa_cheapest_models_examples/', f'aa_cheapest_models_{bench}_above_{performance_lower_bound}_with_trendline')
fig.show()

### Final selected example

In [31]:
# selected_benchmarks = ['MMLU', 'MATH 5', 'HumanEval']
bench = 'GPQA Diamond'
threshold_model = 'GPT-4'
performance_lower_bound = aa_df[aa_df['Model Name'] == threshold_model].iloc[0][bench]
performance_upper_bound = 100
print(f'\nPerformance range: {performance_lower_bound}-{performance_upper_bound}%')
ts = pd.date_range(start='2020-01-01', end=pd.Timestamp.today(), freq='MS')
cheapest_models = []
current_best = None

for i, t in enumerate(ts):
    # Get models published in this time window
    benchmark_df = aa_df
    if i > 0:
        prev_t = ts[i-1]
        benchmark_df = benchmark_df[(benchmark_df['Release Date'] >= prev_t) & (benchmark_df['Release Date'] < t)]
    else:
        benchmark_df = benchmark_df[benchmark_df['Release Date'] < t]
        
    # Filter for performance
    benchmark_df = benchmark_df[
        benchmark_df[bench].notna() &
        (benchmark_df[bench] >= performance_lower_bound) &
        (benchmark_df[bench] < performance_upper_bound)
    ]
    
    if not benchmark_df.empty:
        # Find cheapest new model
        new_best = benchmark_df.loc[benchmark_df['USD per 1M Tokens'].idxmin()]
        
        # Update current best if new model is cheaper (or if no current best)
        if current_best is None or new_best['USD per 1M Tokens'] < current_best['USD per 1M Tokens']:
            current_best = new_best
            cheapest_models.append(current_best)
            print(t, current_best['Model Name'], current_best[bench], f"${current_best['USD per 1M Tokens']:.2f}")

cheapest_models_df = pd.DataFrame(cheapest_models)

# Fit a line to the data
cheapest_models_df['price'] = cheapest_models_df['USD per 1M Tokens']
cheapest_models_df['log_price'] = np.log10(cheapest_models_df['USD per 1M Tokens'])
cheapest_models_df['date'] = cheapest_models_df['Release Date'].map(lambda x: pd.Timestamp(x).toordinal())
exponential_model = smf.ols('log_price ~ date', data=cheapest_models_df).fit()

fig = go.Figure()

all_df = aa_df[
    (aa_df[bench].notna()) &
    (aa_df[bench] >= performance_lower_bound) &
    (aa_df[bench] < performance_upper_bound)
]
fig.add_trace(go.Scatter(
    x=all_df['Release Date'],
    y=all_df['USD per 1M Tokens'],
    mode='markers',
    name=f'Other, GPT-4 level or better on {bench}',
    text=all_df['Model Name'],
    marker=dict(color='rgb(222, 222, 255)')
))

factor = cheapest_models_df['USD per 1M Tokens'].iloc[0] / cheapest_models_df['USD per 1M Tokens'].iloc[-1]
period_months = (cheapest_models_df['Release Date'].iloc[-1] - cheapest_models_df['Release Date'].iloc[0]).days / (365/12)

annual_slope = exponential_model.params['date'] * 365  # Convert daily to annual
annual_factor = int(round(10**(-annual_slope)))  # Convert log slope to factor
# Plot the exponential trendline with the data
date_range = pd.date_range(start=cheapest_models_df['Release Date'].min(), end=cheapest_models_df['Release Date'].max(), freq='MS')
pred_df = pd.DataFrame({'date': date_range.map(lambda x: pd.Timestamp(x).toordinal())})
fig.add_trace(go.Scatter(
    x=date_range,
    y=10**exponential_model.predict(pred_df['date']),
    mode='lines+text',
    name=f'{annual_factor}x decrease per year',
    # Only show text at middle index
    text=['' if i != len(date_range)//2 else f'{annual_factor}x per year' for i in range(len(date_range))],
    textposition='middle left',
    textfont=dict(size=14),
    line=dict(color='rgb(190, 190, 210)', dash='dash'),
    hoverinfo='skip',
    showlegend=False,
))

# annotations = [""] * len(cheapest_models_df['Model Name'])
# annotations[0] = cheapest_models_df['Model Name'].iloc[0]
# annotations[-1] = cheapest_models_df['Model Name'].iloc[-1]
annotations = cheapest_models_df['Model Name']

fig.add_trace(go.Scatter(
    x=cheapest_models_df['Release Date'],
    y=cheapest_models_df['USD per 1M Tokens'],
    mode='markers+text',
    name=f'Cheapest, GPT-4 level or better on {bench}',
    text=annotations,
    textposition='top right',
    # marker=dict(color='blue'),
    # visible="legendonly",  # Hide this trace by default
    # legendgroup=f'{bench}_{performance_range_str}',
))
fig.update_layout(xaxis_range=[
    cheapest_models_df['Release Date'].min() - pd.Timedelta(days=30),
    cheapest_models_df['Release Date'].max() + pd.Timedelta(days=150)
])

fig.update_layout(
    # title=f'The cost to answer PhD-level science questions has fallen by {round(factor, -1):.0f}x in {period_months:.0f} months'
    title=f'The cost to answer PhD-level science questions as well as GPT-4 has fallen by {annual_factor}x per year'
)
fig.update_layout(yaxis_type='log')
fig.update_layout(xaxis_title='Month')
fig.update_layout(yaxis_title='Price in USD per million tokens')
# fig.update_layout(xaxis_range=[aa_df['Release Date'].min() - pd.Timedelta(days=150), aa_df['Release Date'].max() + pd.Timedelta(days=30)])
fig.update_layout(
    width=1000,
    height=600,
    # font=dict(size=14),
    legend=dict(
        yanchor="top",
        y=0.14,
        xanchor="right",
        x=0.45,
        bordercolor="lightgrey",
        borderwidth=1
    )
)

if save:
    save_plot(fig, results_subdir, f'aa_cheapest_models_{bench}_{performance_lower_bound}_to_{performance_upper_bound}', extensions=['png'])

fig.show()


Performance range: 33.0-100%
2023-04-01 00:00:00 GPT-4 33.0 $37.50
2023-12-01 00:00:00 GPT-4 Turbo 50.0 $15.00
2024-03-01 00:00:00 Mistral-Large-2024-02 36.0 $6.00
2024-04-01 00:00:00 Claude-3-Haiku 33.0 $0.50
2024-06-01 00:00:00 Gemini-1.5-Flash-2024-05 39.0 $0.13
2025-02-01 00:00:00 Phi 4 53.0 $0.12


### Full analysis

In [21]:
results_subdir = results_dir + 'cheapest_models_above_previous_frontier/'
os.makedirs(results_subdir, exist_ok=True)

performance_delta = 100
results = []
for i, bench in enumerate(benchmarks):
    if i > 0:
        print('\n')
    print(f'{bench}')

    for i, frontier_model_data in top_models_df_lookup[bench].iterrows():
        performance_lower_bound = frontier_model_data[bench]
        if pd.isna(performance_lower_bound):
            print(f'Frontier model {frontier_model_data["Model Name"]} is missing a {bench} value - skipping')
            continue
        elif benchmark_is_mqa[bench] and performance_lower_bound < 30:
            print(f'Frontier model {frontier_model_data["Model Name"]} has a {bench} value of less than 30% on an MQA benchmark - skipping')
            continue
        if bench == 'LMSys Chatbot Arena ELO':
            performance_upper_bound = np.inf
        else:
            performance_upper_bound = min(performance_lower_bound + performance_delta, 100)
        print(f'\nPerformance range: {performance_lower_bound} ({frontier_model_data["Model Name"]} level) to {performance_upper_bound}')

        ts = pd.date_range(start='2020-01-01', end=pd.Timestamp.today(), freq='MS')
        cheapest_models = []
        current_best = None
        for i, t in enumerate(ts):
            # Get models published in this time window
            benchmark_df = aa_df
            if i > 0:
                prev_t = ts[i-1]
                benchmark_df = benchmark_df[(benchmark_df['Release Date'] >= prev_t) & (benchmark_df['Release Date'] < t)]
            else:
                benchmark_df = benchmark_df[benchmark_df['Release Date'] < t]
                
            # Filter for performance
            benchmark_df = benchmark_df[
                benchmark_df[bench].notna() &
                (benchmark_df[bench] >= performance_lower_bound) &
                (benchmark_df[bench] < performance_upper_bound)
            ]
            
            if not benchmark_df.empty:
                # Find cheapest new model
                new_best = benchmark_df.loc[benchmark_df['USD per 1M Tokens'].idxmin()]
                
                # Update current best if new model is cheaper (or if no current best)
                if current_best is None or new_best['USD per 1M Tokens'] < current_best['USD per 1M Tokens']:
                    current_best = new_best
                    cheapest_models.append(current_best)
                    print(t, current_best['Model Name'], current_best[bench], f"${current_best['USD per 1M Tokens']:.2f}")

        cheapest_models_df = pd.DataFrame(cheapest_models)
        if len(cheapest_models_df) < min_num_data_points_for_regression:
            print(f'Less than {min_num_data_points_for_regression} cheapest models found - skipping')
            continue

        # Fit a line to the data
        cheapest_models_df['price'] = cheapest_models_df['USD per 1M Tokens']
        cheapest_models_df['log_price'] = np.log10(cheapest_models_df['USD per 1M Tokens'])
        cheapest_models_df['date'] = cheapest_models_df['Release Date'].map(lambda x: pd.Timestamp(x).toordinal())
        exponential_model = smf.ols('log_price ~ date', data=cheapest_models_df).fit()

        # Calculate annual rate of decrease
        annual_slope = exponential_model.params['date'] * 365  # Convert daily to annual
        annual_factor = int(round(10**(-annual_slope)))  # Convert log slope to factor
        results.append({
            'bench': bench,
            'threshold_model': frontier_model_data["Model Name"],
            'performance_range': [performance_lower_bound, performance_upper_bound],
            'sample_size': len(cheapest_models_df),
            'start_date': cheapest_models_df['Release Date'].min(),
            'end_date': cheapest_models_df['Release Date'].max(),
            'price_reduction_factor_per_year': annual_factor,
            'r_squared': round(exponential_model.rsquared, 2),
        })

        # Plot the exponential trendline with the data
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=cheapest_models_df['Release Date'],
            y=10**exponential_model.predict(cheapest_models_df['date']),
            mode='lines',
            name=f'Trendline: {annual_factor}x decrease per year',
            line=dict(color='lightgrey', dash='dash')
        ))
        fig.add_trace(go.Scatter(
            x=cheapest_models_df['Release Date'],
            y=cheapest_models_df['USD per 1M Tokens'],
            mode='markers+text',
            name=f'Cheapest models with {performance_lower_bound}-{performance_upper_bound} on {bench}',
            text=cheapest_models_df['Model Name'],
            textposition='bottom left',
            marker=dict(color='blue')
            # line=dict(shape='hv'),
        ))
        other_models_df = aa_df[
            aa_df[bench].notna() &
            (aa_df[bench] >= performance_lower_bound) &
            (aa_df[bench] < performance_upper_bound) &
            ~(aa_df['Model Name'].isin(cheapest_models_df['Model Name']))
        ]
        fig.add_trace(go.Scatter(
            x=other_models_df['Release Date'],
            y=other_models_df['USD per 1M Tokens'],
            mode='markers',
            name=f'Other models with {performance_lower_bound}-{performance_upper_bound} on {bench}',
            text=other_models_df['Model Name'],
            marker=dict(color='lightblue')
        ))
        fig.update_layout(
            title=f'The cheapest model with {performance_lower_bound}-{performance_upper_bound} on {bench} has become {annual_factor}x cheaper per year'
        )
        fig.update_traces(textposition='bottom left')
        fig.update_layout(yaxis_type='log')
        fig.update_layout(xaxis_title='Month')
        fig.update_layout(yaxis_title='Price in USD per million tokens')
        # Lower the lower x limit
        min_date = min(cheapest_models_df['Release Date'].min(), other_models_df['Release Date'].min())
        max_date = max(cheapest_models_df['Release Date'].max(), other_models_df['Release Date'].max())
        fig.update_layout(xaxis_range=[min_date - pd.Timedelta(days=90), max_date + pd.Timedelta(days=30)])
        fig.update_layout(
            width=800,
            height=600,
            font=dict(size=10),
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99,
                bordercolor="lightgrey",
                borderwidth=1
            )
        )
        if save:
            save_plot(
                fig,
                results_subdir,
                f'aa_cheapest_models_{bench}_{performance_lower_bound}-{performance_upper_bound}_with_trendline',
                extensions=['png'],
            )

cheapest_model_results_df = pd.DataFrame(results)
cheapest_model_results_df.to_csv(results_subdir + 'cheapest_model_results.csv', index=False)

# Create a summary DataFrame
summary_data = []
for bench in benchmarks:
    if bench == 'LMSys Chatbot Arena ELO':
        continue
    bench_results = cheapest_model_results_df[cheapest_model_results_df['bench'] == bench]
    # Get all performance lower bounds used for this benchmark
    perf_bounds = list(bench_results['performance_range'])
    
    # Calculate geometric mean of price reduction factors
    price_factors = bench_results['price_reduction_factor_per_year'].dropna()
    geomean = np.exp(np.mean(np.log(price_factors))) if len(price_factors) > 0 else np.nan
    
    # Get range of price reduction factors
    factor_range = [price_factors.min(), price_factors.max()] if len(price_factors) > 0 else []
    
    summary_data.append({
        'bench': bench,
        'performance_range': perf_bounds,
        'price_reduction_factor_per_year_geomean': round(geomean),
        'price_reduction_factor_per_year_range': factor_range
    })

cheapest_model_summary_df = pd.DataFrame(summary_data)
cheapest_model_summary_df.to_csv(results_subdir + 'cheapest_model_summary.csv', index=False)

MMLU

Performance range: 43.9 (GPT-3 level) to 100
2021-12-01 00:00:00 GPT-3 43.9 $60.00
2022-10-01 00:00:00 GPT-3 43.9 $20.00
2023-04-01 00:00:00 GPT-3.5 Turbo 68.0 $2.00
2023-08-01 00:00:00 Llama-2-Chat-13B 45.0 $0.56
2024-01-01 00:00:00 Llama 2-7B 45.3 $0.20
2024-05-01 00:00:00 Llama 2-7B 45.3 $0.13
2024-08-01 00:00:00 Llama-3.1-Instruct-8B 71.0 $0.10
2024-10-01 00:00:00 Llama-3.2-Instruct-3B 64.0 $0.08
2024-11-01 00:00:00 Gemini-1.5-Flash-8B 75.0 $0.07

Performance range: 64.8 (GPT-3.5 level) to 100
2022-12-01 00:00:00 GPT-3.5 64.8 $20.00
2023-04-01 00:00:00 GPT-3.5 Turbo 68.0 $2.00
2023-12-01 00:00:00 GPT-3.5 Turbo 68.0 $0.75
2024-04-01 00:00:00 Claude-3-Haiku 71.0 $0.50
2024-06-01 00:00:00 Gemini-1.5-Flash-2024-05 79.0 $0.13
2024-08-01 00:00:00 Llama-3.1-Instruct-8B 71.0 $0.10
2024-11-01 00:00:00 Gemini-1.5-Flash-8B 75.0 $0.07

Performance range: 86.0 (GPT-4 level) to 100
2023-04-01 00:00:00 GPT-4 86.0 $37.50
2023-12-01 00:00:00 GPT-4 Turbo 87.0 $15.00
2024-06-01 00:00:00 Gemini-

In [22]:
all_price_factors = cheapest_model_results_df['price_reduction_factor_per_year']
np.percentile(all_price_factors, [0, 50, 100])

array([  7.,  50., 331.])

In [23]:
# Create figure
fig = go.Figure()

bench_aliases = {
    'MMLU': 'General knowledge',
    'GPQA Diamond': 'Scientific reasoning', 
    'HumanEval': 'Coding',
    'MATH 5': 'Math (harder)',
    'MATH-500': 'Math (easier)',
    'LMSys Chatbot Arena ELO': 'Chatbot Arena',
}

# Add violin plot
fig.add_trace(go.Violin(
    y=np.log10(all_price_factors),
    x0='All benchmarks',
    box_visible=True,
    name='Distribution',
    showlegend=False,
))

# Add points for each benchmark as scatter traces
for bench in benchmarks:
    # if bench == 'MATH-500':
    #     continue

    bench_results = cheapest_model_results_df[cheapest_model_results_df['bench'] == bench]
    price_factors = bench_results['price_reduction_factor_per_year'].dropna()
    bench_factors = np.log10(price_factors)
    bench_ranges = bench_results['threshold_model']
    
    fig.add_trace(go.Scatter(
        y=bench_factors,
        x=[bench_aliases[bench]] * len(bench_factors),
        mode='markers',
        name=bench,
        text=bench_ranges,
        textposition='middle right',
        marker=dict(size=10),
        hovertemplate='Cheapest models that are %{text} level or better',
        showlegend=False,
    ))

# Update layout
fig.update_layout(yaxis_zeroline=False)
fig.update_layout(yaxis_title='Annualized rate of decrease in price per token')
# Change y-axis labels to be powers of 10
tvs = list(range(0, 6))
fig.update_layout(yaxis=dict(tickmode='array', tickvals=tvs, ticktext=[f'{10**tickval:,.0f}x' for tickval in tvs]))
fig.update_layout(
    title='The lowest-priced LLMs above a capability threshold have become much cheaper to use',
    width=1000,
    height=600,
    font=dict(size=14),
    # add some left margin
    margin=dict(l=130),
    # Increase spacing between x values by expanding the range
    xaxis=dict(range=[-0.5, len(benchmarks) + 1.0])
)
if save:
    save_plot(fig, results_subdir, 'aa_cheapest_models_all_violin', extensions=['png'])
fig.show()